### 🧩 Imports

Import all the necessary libraries:
- `json` for reading/writing JSON files.
- `pandas` and `numpy` for data manipulation.
- `random` for generating random synthetic data.
- `IsolationForest` from `sklearn` for anomaly detection.
- `joblib` for saving the trained model.

In [1]:
import json
import pandas as pd
from sklearn.ensemble import IsolationForest
import numpy as np
import random
import joblib

### 📊 Generate Dataset

Create synthetic "normal" network traffic data to train an anomaly detection model. Each sample includes:

- `src_port`: randomly selected from common service ports.
- `dst_port`: a random high port number.
- `packet_size`: typical packet sizes.
- `duration_ms`: duration of the communication.
- `protocol`: randomly selected between TCP and UDP.

This data is saved to `training_data.json` for future use.

In [4]:
COMMON_PORTS = [80, 443, 22, 8080]

def generate_normal_data():
    return {
        "src_port": random.choice(COMMON_PORTS),
        "dst_port": random.randint(1024, 65535),
        "packet_size": random.randint(100, 1500),
        "duration_ms": random.randint(50, 500),
        "protocol": random.choice(["TCP", "UDP"])
    }

dataset = [generate_normal_data() for _ in range(1000)]

import os

os.makedirs("../dataset", exist_ok=True)
with open("../dataset/training_data.json", "w") as f:
    json.dump(dataset, f, indent=2)


In [58]:
with open("../dataset/training_data.json") as f:
    raw_data = json.load(f)

df = pd.DataFrame(raw_data)
display(df)

,src_port,dst_port,packet_size,duration_ms,protocol
0,80,55018,1372,353,UDP
1,80,24336,719,127,UDP
2,22,51008,768,145,UDP
3,80,45954,978,93,TCP
4,80,34586,163,201,TCP
...,...,...,...,...,...
995,80,44771,704,439,UDP
996,22,18197,1053,201,TCP
997,22,17827,617,392,TCP
998,443,34249,1439,480,TCP


### 🧼 Preprocessing Function

Machine learning models like Isolation Forest require **numerical input only**. Any categorical variables, such as the `protocol` column (`TCP`, `UDP`), must be converted into numbers.

We handle this with **one-hot encoding**, using `pd.get_dummies`.

#### 🛠️ Preprocessing Steps:

1. **Identify categorical columns**:
   - In our case, the `protocol` column is categorical (`TCP`, `UDP`).

2. **Use `pd.get_dummies`**:
   - This creates a new binary column for each category.
   - For instance:
     ```
     protocol
     ---------
     TCP   →   protocol_UDP = 0
     UDP   →   protocol_UDP = 1
     ```
   - Setting `drop_first=True` prevents multicollinearity by dropping the first category (`TCP` here), as it can be inferred from the others.

3. **Return a DataFrame with all numerical values**:
   - This is ready for model input.

> ✅ This preprocessing is essential to avoid errors during training and ensure the model can learn from categorical variables.

In [85]:
def preprocess_data(df):
    display(df)
    df = pd.get_dummies(df, columns=["protocol"], drop_first=True)
    display(df)
    return np.array(df)

In [88]:
normal_data = preprocess_data(df)
display(normal_data)

,src_port,dst_port,packet_size,duration_ms,protocol
0,80,55018,1372,353,UDP
1,80,24336,719,127,UDP
2,22,51008,768,145,UDP
3,80,45954,978,93,TCP
4,80,34586,163,201,TCP
...,...,...,...,...,...
995,80,44771,704,439,UDP
996,22,18197,1053,201,TCP
997,22,17827,617,392,TCP
998,443,34249,1439,480,TCP


,src_port,dst_port,packet_size,duration_ms,protocol_UDP
0,80,55018,1372,353,True
1,80,24336,719,127,True
2,22,51008,768,145,True
3,80,45954,978,93,False
4,80,34586,163,201,False
...,...,...,...,...,...
995,80,44771,704,439,True
996,22,18197,1053,201,False
997,22,17827,617,392,False
998,443,34249,1439,480,False


array([[80, 55018, 1372, 353, True],
       [80, 24336, 719, 127, True],
       [22, 51008, 768, 145, True],
       ...,
       [22, 17827, 617, 392, False],
       [443, 34249, 1439, 480, False],
       [8080, 15698, 408, 185, True]], shape=(1000, 5), dtype=object)

### 🤖 Train Isolation Forest

The `IsolationForest` algorithm is an unsupervised model used to detect anomalies. It isolates observations by randomly selecting features and splitting values.

- `n_estimators=100`: number of trees in the forest.
- `contamination=0.01`: assumes 1% of the data is anomalous.
- `random_state=42`: ensures reproducibility.

The model is trained on the preprocessed numerical dataset.

In [91]:
model = IsolationForest(contamination=0.1, random_state=42)
model.fit(normal_data)

IsolationForest(contamination=0.1, random_state=42)

### 💾 Save Trained Model

Save the trained model using `joblib`, which allows for efficient serialization and deserialization. This saved model can be reused later for inference or deployment.


In [94]:
model_info = {
    'model': model, 
    'columns': df.columns.tolist()
}
joblib.dump(model, "anomaly_model.joblib")

['anomaly_model.joblib']

# predict data

In [93]:
print(model.predict(normal_data))

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1 -1  1  1  1  1  1  1  1  1  1  1 -1  1  1 -1 -1  1  1  1  1  1
  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1 -1  1  1 -1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1
  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1 -1  1
  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1 -1  1  1  1  1  1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1 -1  1  1
  1  1  1  1 -1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1
  1  1  1  1 -1 -1  1 -1  1  1  1  1  1  1  1  1  1